# Define Library

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
import tempfile
import time
from datetime import datetime
import uuid
import joblib
import uuid

import gcsfs
import duckdb as dd
import pickle
import joblib
from typing import Union
import io

path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"
# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)

In [5]:
model_Inventory = pd.read_excel(r"D:\OneDrive - Tonik Financial Pte Ltd\MyStuff\Data Engineering\Model_Monitoring\New_Model_Monitoring\Source_Data\Model_Inventory.xlsx", sheet_name='Sheet1')

In [6]:
model_Inventory.head()

,Product Name​,Model Name​,Model Version,AI Scorecard (Y/N),Customer Segment​,Predictive Model Description​,Data Source,Training Period,Target,Current Status,Latest mature month Performance for Target,Model Servicing (Batch/ Real Time),Owner​,Go-Live​ Month
0,"Cash, SIL",Cash/ SIL Demo Scorecard 1.0​,1.0,N,All Cash and SIL Loan applicants​,Predict if the Cash or SIL loan will become FP...,Demographic data captured from loan application,NaN,FPD30,Retired,NaN,NaN,Neel​,Dec 2022​
1,"Cash, SIL",Cash/ SIL Demo Scorecard 2.0​,2.0,N,All Cash and SIL Loan applicants​,Predict if the Cash or SIL loan will become FP...,Demographic data captured from loan application,NaN,FPD30,Retired,NaN,NaN,Marek​,May 2023​
2,Tendo,Tendo Payroll Scorecard 1.0​,1.0,N,All Tendo Customers​,Predict which Tendo customers will become FSPD...,Demographic data captured from loan application,NaN,FPD30,Retired,NaN,NaN,Marek​,Sep 2023​
3,"Cash, SIL",Cash/ SIL CIC Scorecard 1.0​,1.0,N,All Cash and SIL Loan applicants​,Predict if a Cash/SIL loan will become FPD15 w...,Credit Bureau data from CIC,NaN,FPD15,Retired,NaN,NaN,Marek​,Oct 2023​
4,"Cash, SIL",Cash/ SIL Generic Credo Score 1.0​,1.0,N,All Cash and SIL Loan applicants​,Predict if a Cash/SIL loan will become FPD30 w...,Credit Bureau data from CIC,NaN,FPD30,Retired,NaN,NaN,CredoLab​,Dec 2024​


In [7]:
model_Inventory['Model Name​'].value_counts()

Model Name​
Cash/ SIL Demo Scorecard 1.0​                            1
Cash Trench3 Credo Score v1​                             1
SIL Trench1 & Trench2 Apps Score v1​                     1
SIL Trench2 Transaction Score v1​                        1
SIL Beta Demo Score v2​                                  1
Cash Trench3 Alpha Stack v1​                             1
Cash Trench3 Beta Stack v1​                              1
Cash Trench3 Gamma Stack v2​\nAKA\nCash B Score v2       1
Cash Trench3 Apps Score v1​                              1
Cash Trench1 Gamma Stack Score v1​                       1
Cash Trench 2 Alpha Stack v1​                            1
Cash Trench 1 Alpha Stack v1​                            1
Cash Alpha CIC Score v1​                                 1
Cash Trench 2 Beta Stack v1​                             1
Cash Trench 1 Beta Stack v1​                             1
Cash Beta Demo Score v1​                                 1
SIL Trench1 & Trench2 Credo Score v1​       

The above table the model name is different and will need domain knowledge to map it with the model name in existing ml_run_details table.

# `prj-prod-dataplatform.audit_balance.ml_request_details`

In [ ]:
from regex import P


sq = """WITH parsed as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
FROM `prj-prod-dataplatform.audit_balance.ml_request_details`
)
select * from parsed
;"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

# Query

In [ ]:
# sq = """ 
# create or replace table `dap_ds_poweruser_playground.Model_score_flattened_dc` as 
# WITH parsed as (
# select customerId,digitalLoanAccountId,modelName, publish_time,
# REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
# FROM `prj-prod-dataplatform.audit_balance.ml_request_details`
# )
# ,
# latest_request as (
# select * from parsed
# QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId ORDER BY publish_time DESC ) = 1)
# ,
# model_run as (
# select digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time, prediction
# from `prj-prod-dataplatform.audit_balance.ml_model_run_details`)
# select 
#   r.customerId,
#   r.digitalLoanAccountId,
#   r.modelName,
#   m.modelDisplayName,
#   m.modelVersionId,
#   m.start_time,
#   m.end_time,
#   m.prediction,
#   r.publish_time,
#   -- ====== Predictions ======
#   JSON_VALUE(r.requestPayload_clean, "$.predictions.digitalLoanAccountId") AS digitalLoanAccountId_json,
#   JSON_VALUE(r.requestPayload_clean, "$.predictions.crifApplicationId") AS crifApplicationId,
#   SAFE_CAST(JSON_VALUE(r.requestPayload_clean, "$.predictions.customerId") AS INT64) AS customerId_json,
#   SAFE_CAST(JSON_VALUE(r.requestPayload_clean, "$.predictions.bDemoScore") AS FLOAT64) AS bDemoScore,
#   SAFE_CAST(JSON_VALUE(r.requestPayload_clean, "$.predictions.ibEstimatedIncome") AS FLOAT64) AS ibEstimatedIncome,
#   SAFE_CAST(JSON_VALUE(r.requestPayload_clean, "$.predictions.bStackScore") AS FLOAT64) AS bStackScore,
#   SAFE_CAST(JSON_VALUE(r.requestPayload_clean, "$.predictions.credoScore") AS FLOAT64) AS credoScore,
#   SAFE_CAST(JSON_VALUE(r.requestPayload_clean, "$.predictions.bTransactionScore") AS FLOAT64) AS bTransactionScore,
#   SAFE_CAST(JSON_VALUE(r.requestPayload_clean, "$.predictions.appsScore") AS FLOAT64) AS appsScore,
#   JSON_VALUE(r.requestPayload_clean, "$.predictions.trenchCategory") AS trenchCategory,
#   if (loanmaster.new_loan_type ='Flex-up',loanmaster.startApplyDateTime,loanmaster.termsAndConditionsSubmitDateTime) appln_submit_datetime,
# FROM latest_request r
# left join model_run m
# on r.digitalLoanAccountId = m.digitalLoanAccountId
# left join  risk_credit_mis.loan_master_table loanmaster on loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
# ;
# """
# job = client.query(sq)
# job.result()  # Wait for the job to complete.
# time.sleep(5) # Delays for 30 seconds
# print(f'Table dap_ds_poweruser_playground.Model_score_flattened_dc created successfully')


Table dap_ds_poweruser_playground.Model_score_flattened_dc created successfully


The above query is too complicated and is not suitable for PSI, Gini calculation.

In [10]:
sq = """select * from dap_ds_poweruser_playground.Model_score_flattened_dc;"""

dfd = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
print(f":The shape of dap_ds_poweruser_playground.Model_score_flattened_dc is:{dfd.shape}")

Job ID 4eaebecb-6e0d-4fd5-801a-3b8fd4e37c60 successfully executed: 100%|██████████|

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
:The shape of dap_ds_poweruser_playground.Model_score_flattened_dc is:(535952, 20)


In [11]:
dfd.head()

,customerId,digitalLoanAccountId,modelName,modelDisplayName,modelVersionId,start_time,end_time,prediction,publish_time,digitalLoanAccountId_json,crifApplicationId,customerId_json,bDemoScore,ibEstimatedIncome,bStackScore,credoScore,bTransactionScore,appsScore,trenchCategory,appln_submit_datetime
0,3501852,324c7952-1323-4287-a686-d24283a2aa37,SIL - Alpha Model,Beta - DemoScoreModel,v1,2025-06-17 14:46:13.912941,2025-06-17 14:46:13.979002,0.0893572768,2025-06-16 11:15:19.853000+00:00,None,None,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,None,2025-06-16 19:15:10
1,2463443,8e5ef6c3-ccb3-4bb8-adaa-82c7ccfbd7eb,SIL - Alpha Model,Beta - IncomeEstimationModel,v1,2025-08-29 01:02:58.787282,2025-08-29 01:02:58.859220,8892.488166260635,2025-08-29 01:03:07.748000+00:00,None,None,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,None,2025-08-29 09:02:54
2,3523221,449983eb-e87a-461d-9349-c3aa5859dd4a,SIL - Alpha Model,Beta - AppsScoreModel,v1,2025-06-27 07:29:22.761041,2025-06-27 07:29:23.222806,"{'cb_score': 0.34780714404654456, 'dl_score': ...",2025-06-27 07:29:37.910000+00:00,None,None,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,None,2025-06-27 15:29:20
3,3659208,864f1d67-8a28-4567-b32a-7c8c56717914,SIL - Alpha Model,Alpha - CIC-SIL-Model,v1,2025-09-02 05:38:38.738207,2025-09-02 05:38:38.744622,0.16215553639025374,2025-09-02 05:38:38.885000+00:00,None,None,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,None,2025-09-02 13:38:30
4,3572166,808b3d0c-8716-4041-9668-d1c5f49581dc,SIL - Alpha Model,Alpha - StackingModel,v1,2025-07-22 08:13:59.836241,2025-07-22 08:13:59.842547,0.04004346416278096,2025-07-22 08:13:59.905000+00:00,None,None,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,None,2025-07-22 16:13:52


In [12]:
dfd[['modelDisplayName', 'trenchCategory']].value_counts(dropna=False)

modelDisplayName                trenchCategory
Beta - IncomeEstimationModel    NaN               89332
Beta - DemoScoreModel           NaN               89332
Beta - StackScoreModel          NaN               89331
Beta - AppsScoreModel           NaN               79129
Alpha  - IncomeEstimationModel  NaN               56996
Alpha - StackingModel           NaN               56996
Alpha - CIC-SIL-Model           NaN               56996
NaN                             NaN               10628
Beta-Cash-Demo-Model            NaN                1401
Beta-Cash-Stack-Model           NaN                1401
Beta-Cash-AppScore-Model        NaN                 943
Beta-Cash-Demo-Model            Trench 1            710
Beta-Cash-Stack-Model           Trench 1            710
Beta-Cash-AppScore-Model        Trench 1            477
Alpha-Cash-Stack-Model          NaN                 398
Alpha-Cash-CIC-Model            NaN                 398
Beta-Cash-Demo-Model            Trench 2            211
B

# PSI

`We will create for each model separate PSI file and add it to a common table. If there is any new model in future we will add it individually`

In [21]:
# sq = """
# create or replace table `dap_ds_poweruser_playground.Model_score_flattened_PSI_dc`
# SELECT distinct
#   a1.customerId, 
#   a1.digitalLoanAccountId,
#   a1.modelDisplayName,
#   a1.modelVersionId,
# --  a1.calcFeature,
#   a1.prediction,
  
# #   -- Extract each field from calcFeature JSON
# #   JSON_VALUE(a1.calcFeature, '$.ln_vas_opted_flag') AS ln_vas_opted_flag,
# #   CAST(JSON_VALUE(a1.calcFeature, '$.ln_self_dec_income') AS INT64) AS ln_self_dec_income,
# #   CAST(JSON_VALUE(a1.calcFeature, '$.ln_age') AS INT64) AS ln_age,
# #   JSON_VALUE(a1.calcFeature, '$.ln_source_funds_new_bin') AS ln_source_funds_new_bin,
# #   JSON_VALUE(a1.calcFeature, '$.ln_loan_level_user_type') AS ln_loan_level_user_type,
# #   JSON_VALUE(a1.calcFeature, '$.ln_industry_new_cat_bin') AS ln_industry_new_cat_bin,
# #   JSON_VALUE(a1.calcFeature, '$.ln_marital_status') AS ln_marital_status,
# #   JSON_VALUE(a1.calcFeature, '$.ln_doc_type_rolled') AS ln_doc_type_rolled,
# #   JSON_VALUE(a1.calcFeature, '$.ln_education_level') AS ln_education_level,
# #   JSON_VALUE(a1.calcFeature, '$.ln_ref2_type') AS ln_ref2_type,
# #   JSON_VALUE(a1.calcFeature, '$.ln_email_primary_domain') AS ln_email_primary_domain,
# #   JSON_VALUE(a1.calcFeature, '$.ln_os_type') AS ln_os_type,
# #   JSON_VALUE(a1.calcFeature, '$.ln_mature_fspd30_flag') AS ln_mature_fspd30_flag,
# #   JSON_VALUE(a1.calcFeature, '$.ln_fspd30_flag') AS ln_fspd30_flag,
# #   CAST(JSON_VALUE(a1.calcFeature, '$.ln_appln_submit_datetime') AS TIMESTAMP) AS ln_appln_submit_datetime,
# #   JSON_VALUE(a1.calcFeature, '$.ln_province_bin') AS ln_province_bin,
# if (loanmaster.new_loan_type ='Flex-up',loanmaster.startApplyDateTime,loanmaster.termsAndConditionsSubmitDateTime) appln_submit_datetime,
# loanmaster.disbursementDateTime
# FROM 
#   `prj-prod-dataplatform.audit_balance.ml_model_run_details` a1
# left join  risk_credit_mis.loan_master_table loanmaster on loanmaster.digitalLoanAccountId = a1.digitalLoanAccountId
# ;
# """

sq = """ 
create or replace table `dap_ds_poweruser_playground.Model_score_flattened_PSI_dc` as
SELECT DISTINCT
  a1.customerId, 
  a1.digitalLoanAccountId,
  a1.modelDisplayName,
  a1.modelVersionId,
  -- Extract combined_score if prediction is JSON, otherwise use the original value
  CASE 
    WHEN a1.prediction LIKE '{%' THEN CAST(JSON_VALUE(a1.prediction, '$.combined_score') AS FLOAT64)
    ELSE CAST(a1.prediction AS FLOAT64)
  END AS prediction_score,
  
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime
FROM 
  `prj-prod-dataplatform.audit_balance.ml_model_run_details` a1
LEFT JOIN risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = a1.digitalLoanAccountId
;
"""

job = client.query(sq)
job.result()  # Wait for the job to complete.
time.sleep(5) # Delays for 30 seconds
print(f'Table `dap_ds_poweruser_playground.Model_score_flattened_PSI_dc` created successfully')

sq = """select * from `dap_ds_poweruser_playground.Model_score_flattened_PSI_dc`;"""

df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
print(f"The shape of the dataframe is:{df.shape}")
df.head()

Table `dap_ds_poweruser_playground.Model_score_flattened_PSI_dc` created successfully
Job ID aa66eeaa-ea04-48b4-9911-c19fedff24dd successfully executed: 100%|██████████|

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of the dataframe is:(542279, 7)


,customerId,digitalLoanAccountId,modelDisplayName,modelVersionId,prediction_score,appln_submit_datetime,disbursementDateTime
0,1007181,5ec62744-5047-49ea-8b88-b6c8b4f66ca0,Beta-Cash-Stack-Model,v1,0.133093,NaT,NaT
1,1007181,5ec62744-5047-49ea-8b88-b6c8b4f66ca0,Beta-Cash-AppScore-Model,v1,0.356434,NaT,NaT
2,1007181,5ec62744-5047-49ea-8b88-b6c8b4f66ca0,Beta-Cash-Demo-Model,v1,0.461388,NaT,NaT
3,1007697,ef2b55c3-1563-4cac-907e-d4d3ff0d108b,Beta-Cash-Stack-Model,v1,0.132082,2025-10-03 01:11:22,NaT
4,1007697,ef2b55c3-1563-4cac-907e-d4d3ff0d108b,Beta-Cash-Demo-Model,v1,0.354306,2025-10-03 01:11:22,NaT


In [22]:
df['modelDisplayName'].value_counts(dropna=False)

modelDisplayName
Beta - DemoScoreModel             92633
Beta - StackScoreModel            92632
Beta - IncomeEstimationModel      92630
Beta - AppsScoreModel             82095
Alpha - CIC-SIL-Model             58350
Alpha - StackingModel             58350
Alpha  - IncomeEstimationModel    58348
Beta-Cash-Stack-Model              2417
Beta-Cash-Demo-Model               2416
Beta-Cash-AppScore-Model           1618
Alpha-Cash-CIC-Model                395
Alpha-Cash-Stack-Model              395
Name: count, dtype: int64

In [24]:
dfd = df[['digitalLoanAccountId', 'customerId']].copy()
dd.query("""select digitalLoanAccountId, count(digitalLoanAccountId) from dfd group by 1 having count(digitalLoanAccountId) > 1 order by 2 desc;""").to_df()

,digitalLoanAccountId,count(digitalLoanAccountId)
0,e3466624-a586-4ea7-aa89-d5bc98ae7eca,10
1,7f3f9081-1050-43cd-8bdc-40ad8c69b142,9
2,6be790f2-d234-43cb-90a7-0532ae970a62,8
3,0c73d1e0-dad2-41d5-aa03-08f73ddef75b,8
4,bd00d9a6-fc19-44c5-beaf-b613c2d73756,8
...,...,...
95041,477e9b49-af98-4e81-8339-796c4b5061e7,2
95042,8a143a5e-55f5-4e4f-bb22-b904bf03c83a,2
95043,b13df02a-7cd2-4c98-965a-daab62629f9a,2
95044,154f68d6-e38e-4e17-bb35-a9d61de32168,2


In [25]:
df[df['digitalLoanAccountId'] == 'e3466624-a586-4ea7-aa89-d5bc98ae7eca']

,customerId,digitalLoanAccountId,modelDisplayName,modelVersionId,prediction_score,appln_submit_datetime,disbursementDateTime
18761,3628558,e3466624-a586-4ea7-aa89-d5bc98ae7eca,Alpha - IncomeEstimationModel,v1,8994.518246,2025-08-18 14:31:27,NaT
77109,3628558,e3466624-a586-4ea7-aa89-d5bc98ae7eca,Alpha - CIC-SIL-Model,v1,0.057885,2025-08-18 14:31:27,NaT
77110,3628558,e3466624-a586-4ea7-aa89-d5bc98ae7eca,Alpha - CIC-SIL-Model,v1,0.141446,2025-08-18 14:31:27,NaT
135459,3628558,e3466624-a586-4ea7-aa89-d5bc98ae7eca,Alpha - StackingModel,v1,0.167063,2025-08-18 14:31:27,NaT
135460,3628558,e3466624-a586-4ea7-aa89-d5bc98ae7eca,Alpha - StackingModel,v1,0.083470,2025-08-18 14:31:27,NaT
200323,3628558,e3466624-a586-4ea7-aa89-d5bc98ae7eca,Beta - AppsScoreModel,v1,0.643616,2025-08-18 14:31:27,NaT
284554,3628558,e3466624-a586-4ea7-aa89-d5bc98ae7eca,Beta - DemoScoreModel,v1,0.017603,2025-08-18 14:31:27,NaT
284555,3628558,e3466624-a586-4ea7-aa89-d5bc98ae7eca,Beta - DemoScoreModel,v1,0.017381,2025-08-18 14:31:27,NaT
377187,3628558,e3466624-a586-4ea7-aa89-d5bc98ae7eca,Beta - IncomeEstimationModel,v1,8198.067159,2025-08-18 14:31:27,NaT
469817,3628558,e3466624-a586-4ea7-aa89-d5bc98ae7eca,Beta - StackScoreModel,v1,0.116758,2025-08-18 14:31:27,NaT


In [ ]:
# import pandas as pd
# import numpy as np
# from datetime import datetime

# # First, let's check the datetime column format
# print("Sample datetime values:")
# print(df['appln_submit_datetime'].head(10))
# print(f"\nData type: {df['appln_submit_datetime'].dtype}")

# # Try to convert datetime - handle different formats
# try:
#     # If it's already datetime
#     if pd.api.types.is_datetime64_any_dtype(df['appln_submit_datetime']):
#         df['month_apply_submit_date'] = df['appln_submit_datetime'].dt.to_period('M')
#     else:
#         # Try different date formats
#         df['month_apply_submit_date'] = pd.to_datetime(
#             df['appln_submit_datetime'], 
#             format='mixed',
#             errors='coerce'
#         ).dt.to_period('M')
# except Exception as e:
#     print(f"Error converting dates: {e}")
#     # If all else fails, try inferring format
#     df['month_apply_submit_date'] = pd.to_datetime(
#         df['appln_submit_datetime'],
#         infer_datetime_format=True,
#         errors='coerce'
#     ).dt.to_period('M')

# # Remove rows with NaT dates
# df_clean = df.dropna(subset=['month_apply_submit_date']).copy()
# print(f"\nRecords with valid dates: {len(df_clean)} out of {len(df)}")

# if len(df_clean) == 0:
#     print("\nERROR: No valid dates found. Please check the format of 'ln_appln_submit_datetime' column.")
# else:
#     # Create 10 bins for prediction scores
#     df_clean['prediction'] = pd.to_numeric(df_clean['prediction'], errors='coerce')
#     df_clean = df_clean.dropna(subset=['prediction'])
    
#     # Create bins
#     df_clean['prediction_bin'] = pd.cut(df_clean['prediction'], bins=10, labels=False)
    
#     # Function to calculate PSI between two distributions
#     def calculate_psi(expected_dist, actual_dist, bins=10):
#         """
#         Calculate PSI between expected (baseline) and actual distributions
#         """
#         all_bins = range(bins)
        
#         # Get proportions for each bin
#         expected = np.array([expected_dist.get(bin, 0.0001) for bin in all_bins])
#         actual = np.array([actual_dist.get(bin, 0.0001) for bin in all_bins])
        
#         # Ensure they sum to 1
#         expected = expected / expected.sum()
#         actual = actual / actual.sum()
        
#         # Calculate PSI
#         psi = np.sum((actual - expected) * np.log(actual / expected))
        
#         return psi
    
#     # Calculate PSI for each model
#     results = []
    
#     for model in df_clean['modelDisplayName'].unique():
#         model_df = df_clean[df_clean['modelDisplayName'] == model].copy()
        
#         # Sort by month
#         months = sorted(model_df['month_apply_submit_date'].unique())
        
#         if len(months) < 2:
#             print(f"\nWarning: Model '{model}' has only {len(months)} month(s) of data. Need at least 2 for month-on-month PSI.")
#             continue
        
#         # Use first month as baseline
#         baseline_month = months[0]
#         baseline_df = model_df[model_df['month_apply_submit_date'] == baseline_month]
#         baseline_dist = baseline_df['prediction_bin'].value_counts(normalize=True).sort_index()
        
#         print(f"\n{'='*60}")
#         print(f"Model: {model}")
#         print(f"Baseline month: {baseline_month}")
#         print(f"{'='*60}")
        
#         # Calculate PSI for each month compared to baseline
#         for month in months:
#             month_df = model_df[model_df['month_apply_submit_date'] == month]
            
#             if len(month_df) == 0:
#                 continue
            
#             # Calculate actual distribution for the month
#             actual_dist = month_df['prediction_bin'].value_counts(normalize=True).sort_index()
            
#             # Calculate PSI
#             psi_value = calculate_psi(baseline_dist, actual_dist)
            
#             account_count = month_df['digitalLoanAccountId'].nunique()
            
#             results.append({
#                 'modelDisplayName': model,
#                 'month': str(month),
#                 'baseline_month': str(baseline_month),
#                 'PSI': round(psi_value, 6),
#                 'account_count': account_count,
#                 'sample_size': len(month_df)
#             })
    
#     # Create results dataframe
#     psi_results_df = pd.DataFrame(results)
    
#     if len(psi_results_df) > 0:
#         print("\n" + "="*80)
#         print("PSI RESULTS (Month-on-Month)")
#         print("="*80)
#         print(psi_results_df.to_string(index=False))
        
#         # PSI Interpretation
#         print("\n" + "="*80)
#         print("PSI INTERPRETATION GUIDE:")
#         print("="*80)
#         print("PSI < 0.1  : No significant change (Green)")
#         print("0.1 ≤ PSI < 0.25 : Moderate change (Amber) - Monitor closely")
#         print("PSI ≥ 0.25 : Significant change (Red) - Action required")
        
#         # Flag high PSI values
#         high_psi = psi_results_df[psi_results_df['PSI'] >= 0.1]
#         if len(high_psi) > 0:
#             print(f"\n⚠️  WARNING: {len(high_psi)} month(s) with PSI ≥ 0.1 detected:")
#             print(high_psi[['modelDisplayName', 'month', 'PSI', 'account_count']].to_string(index=False))
        
#         # Summary statistics
#         print("\n" + "="*80)
#         print("SUMMARY STATISTICS:")
#         print("="*80)
#         print(f"Total records processed: {len(df_clean)}")
#         print(f"Unique models: {df_clean['modelDisplayName'].nunique()}")
#         print(f"Date range: {df_clean['month_apply_submit_date'].min()} to {df_clean['month_apply_submit_date'].max()}")
#         print(f"Number of months: {df_clean['month_apply_submit_date'].nunique()}")
#         print(f"\nAverage PSI: {psi_results_df['PSI'].mean():.4f}")
#         print(f"Max PSI: {psi_results_df['PSI'].max():.4f}")
#         print(f"Min PSI: {psi_results_df['PSI'].min():.4f}")
        
#         # Display the dataframe for further use
#         display(psi_results_df)
#     else:
#         print("\nNo PSI results generated. Please check your data.")

In [30]:
import pandas as pd
import numpy as np
from datetime import datetime

def calculate_psi_with_smoothing(expected_dist, actual_dist, bins=10, epsilon=0.0001):
    """Calculate PSI with smoothing to handle zero values"""
    all_bins = range(bins)
    expected = np.array([expected_dist.get(bin, epsilon) for bin in all_bins])
    actual = np.array([actual_dist.get(bin, epsilon) for bin in all_bins])
    
    # Normalize
    expected = expected / expected.sum()
    actual = actual / actual.sum()
    
    # Calculate PSI
    psi = np.sum((actual - expected) * np.log(actual / expected))
    return psi

def calculate_monthly_psi(df):
    """
    Calculate monthly PSI for model monitoring using quantile-based binning
    """
    # Create a copy to avoid modifying original dataframe
    df_clean = df.copy()
    
    # Debug: Check initial data
    print("Initial data overview:")
    print(f"Total records: {len(df_clean)}")
    print(f"Columns: {df_clean.columns.tolist()}")
    print(f"Models: {df_clean['modelDisplayName'].unique()}")
    
    # Use correct column name 'prediction_score'
    prediction_col = 'prediction_score'
    
    # Check prediction column
    print(f"\nPrediction column info:")
    print(f"Data type: {df_clean[prediction_col].dtype}")
    print(f"Non-null values: {df_clean[prediction_col].notna().sum()}")
    print(f"Null values: {df_clean[prediction_col].isna().sum()}")
    
    print(f"Prediction sample values: {df_clean[prediction_col].head().tolist()}")
    print(f"Prediction range: {df_clean[prediction_col].min()} - {df_clean[prediction_col].max()}")
    print(f"Prediction mean: {df_clean[prediction_col].mean()}")
    print(f"Prediction median: {df_clean[prediction_col].median()}")
    
    # Use appln_submit_datetime which already exists in the DataFrame
    datetime_column = 'appln_submit_datetime'
    
    # Convert datetime and create month period
    print("\nConverting datetime...")
    df_clean['month_apply_submit_date'] = pd.to_datetime(
        df_clean[datetime_column], errors='coerce'
    ).dt.to_period('M')
    
    # Remove rows with invalid dates
    initial_count = len(df_clean)
    df_clean = df_clean.dropna(subset=['month_apply_submit_date'])
    print(f"Removed {initial_count - len(df_clean)} records with invalid dates")
    
    if len(df_clean) == 0:
        print("ERROR: No valid dates after cleaning")
        return pd.DataFrame()
    
    # Convert prediction to numeric and remove invalid values
    print("Processing predictions...")
    df_clean['prediction_numeric'] = pd.to_numeric(df_clean[prediction_col], errors='coerce')
    initial_count2 = len(df_clean)
    df_clean = df_clean.dropna(subset=['prediction_numeric'])
    print(f"Removed {initial_count2 - len(df_clean)} records with invalid predictions")
    
    if len(df_clean) == 0:
        print("ERROR: No valid predictions after cleaning")
        return pd.DataFrame()
    
    print(f"Valid predictions range: {df_clean['prediction_numeric'].min():.6f} - {df_clean['prediction_numeric'].max():.6f}")
    print(f"Valid predictions mean: {df_clean['prediction_numeric'].mean():.6f}")
    
    # Calculate PSI for each model
    results = []
    
    for model in df_clean['modelDisplayName'].unique():
        model_df = df_clean[df_clean['modelDisplayName'] == model].copy()
        months = sorted(model_df['month_apply_submit_date'].unique())
        
        print(f"\n{'='*80}")
        print(f"Processing model: {model}")
        print(f"{'='*80}")
        print(f"Months available: {[str(m) for m in months]}")
        print(f"Records per month:")
        for month in months:
            month_count = len(model_df[model_df['month_apply_submit_date'] == month])
            print(f"  {month}: {month_count} records")
        
        if len(months) < 2:
            print(f"Warning: Model '{model}' has only {len(months)} month(s). Need at least 2 for PSI calculation.")
            continue
        
        # Use first month as baseline
        baseline_month = months[0]
        baseline_df = model_df[model_df['month_apply_submit_date'] == baseline_month]
        
        print(f"\nBaseline month: {baseline_month}, Records: {len(baseline_df)}")
        print(f"Baseline prediction stats:")
        print(f"  Min: {baseline_df['prediction_numeric'].min():.6f}")
        print(f"  Max: {baseline_df['prediction_numeric'].max():.6f}")
        print(f"  Mean: {baseline_df['prediction_numeric'].mean():.6f}")
        print(f"  Median: {baseline_df['prediction_numeric'].median():.6f}")
        
        # CRITICAL FIX: Create quantile-based bins using baseline month
        # This ensures equal-frequency bins instead of equal-width bins
        try:
            # Create deciles (10 bins with equal number of observations)
            quantile_bins = baseline_df['prediction_numeric'].quantile(
                [i/10 for i in range(11)]
            ).values
            
            # Handle duplicate quantile values
            quantile_bins = np.unique(quantile_bins)
            
            if len(quantile_bins) < 3:
                print(f"Warning: Not enough unique values for binning in baseline. Skipping model.")
                continue
            
            # Extend bins to cover full range of all data for this model
            quantile_bins[0] = model_df['prediction_numeric'].min() - 0.0001
            quantile_bins[-1] = model_df['prediction_numeric'].max() + 0.0001
            
            print(f"\nQuantile bin edges: {quantile_bins}")
            
            # Apply bins to baseline
            baseline_df['prediction_bin'] = pd.cut(
                baseline_df['prediction_numeric'],
                bins=quantile_bins,
                labels=False,
                include_lowest=True,
                duplicates='drop'
            )
            
            baseline_dist = baseline_df['prediction_bin'].value_counts(normalize=True)
            
            print(f"Baseline distribution across bins:")
            for bin_idx in sorted(baseline_dist.index):
                print(f"  Bin {bin_idx}: {baseline_dist[bin_idx]:.4f} ({int(baseline_dist[bin_idx] * len(baseline_df))} records)")
            
        except Exception as e:
            print(f"Error creating quantile bins for {model}: {e}")
            continue
        
        # Calculate PSI for each month using the same bins
        for month in months:
            month_df = model_df[model_df['month_apply_submit_date'] == month].copy()
            
            if len(month_df) == 0:
                continue
            
            # Apply the same quantile bins to this month's data
            month_df['prediction_bin'] = pd.cut(
                month_df['prediction_numeric'],
                bins=quantile_bins,
                labels=False,
                include_lowest=True,
                duplicates='drop'
            )
            
            # Calculate actual distribution for the month
            actual_dist = month_df['prediction_bin'].value_counts(normalize=True)
            
            # Calculate PSI
            psi_value = calculate_psi_with_smoothing(baseline_dist, actual_dist, bins=len(quantile_bins)-1)
            
            account_count = month_df['digitalLoanAccountId'].nunique()
            
            results.append({
                'modelDisplayName': model,
                'month': str(month),
                'baseline_month': str(baseline_month),
                'PSI': round(psi_value, 6),
                'account_count': account_count,
                'sample_size': len(month_df),
                'prediction_min': month_df['prediction_numeric'].min(),
                'prediction_max': month_df['prediction_numeric'].max(),
                'prediction_mean': month_df['prediction_numeric'].mean(),
                'num_bins': len(quantile_bins) - 1
            })
            
            print(f"\nMonth {month}:")
            print(f"  PSI = {psi_value:.6f}")
            print(f"  Records = {len(month_df)}")
            print(f"  Prediction stats: min={month_df['prediction_numeric'].min():.2f}, max={month_df['prediction_numeric'].max():.2f}, mean={month_df['prediction_numeric'].mean():.2f}")
    
    return pd.DataFrame(results)

def interpret_psi_results(psi_results_df):
    """
    Interpret and display PSI results with recommendations
    """
    if len(psi_results_df) == 0:
        print("No PSI results to interpret.")
        return
    
    print("\n" + "="*80)
    print("PSI RESULTS INTERPRETATION")
    print("="*80)
    
    # PSI Interpretation Guide
    print("\nPSI INTERPRETATION GUIDE:")
    print("PSI < 0.1    : No significant population shift (Green)")
    print("0.1 ≤ PSI < 0.25 : Moderate population shift (Amber) - Monitor closely")
    print("PSI ≥ 0.25   : Significant population shift (Red) - Investigation required")
    
    # Categorize PSI values
    psi_results_df['psi_category'] = pd.cut(
        psi_results_df['PSI'],
        bins=[-0.001, 0.1, 0.25, float('inf')],
        labels=['Green', 'Amber', 'Red']
    )
    
    # Display results
    print("\nDetailed Results:")
    display_cols = ['modelDisplayName', 'month', 'baseline_month', 'PSI', 'psi_category', 'account_count', 'sample_size']
    print(psi_results_df[display_cols].to_string(index=False))
    
    # Flag alerts
    high_psi = psi_results_df[psi_results_df['PSI'] >= 0.1]
    if len(high_psi) > 0:
        print(f"\n⚠️  ALERTS: {len(high_psi)} instance(s) with PSI ≥ 0.1 detected:")
        alert_cols = ['modelDisplayName', 'month', 'PSI', 'psi_category', 'account_count']
        print(high_psi[alert_cols].to_string(index=False))
    else:
        print("\n✅ No significant population shifts detected (All PSI < 0.1)")
    
    # Summary statistics
    print("\n" + "="*80)
    print("SUMMARY STATISTICS")
    print("="*80)
    print(f"Total models analyzed: {psi_results_df['modelDisplayName'].nunique()}")
    print(f"Total month comparisons: {len(psi_results_df)}")
    print(f"Average PSI: {psi_results_df['PSI'].mean():.6f}")
    print(f"Maximum PSI: {psi_results_df['PSI'].max():.6f}")
    print(f"Minimum PSI: {psi_results_df['PSI'].min():.6f}")
    
    # Distribution of categories
    category_counts = psi_results_df['psi_category'].value_counts()
    print(f"\nPSI Category Distribution:")
    for category, count in category_counts.items():
        print(f"  {category}: {count} instance(s)")

# Execute the analysis
print("Starting PSI Analysis with Quantile-Based Binning...")
print("="*80)

# Calculate PSI
psi_results_df = calculate_monthly_psi(df)

if len(psi_results_df) > 0:
    # Interpret results
    interpret_psi_results(psi_results_df)
    
    # Optional: Save results to CSV
    output_file = "psi_monitoring_results.csv"
    psi_results_df.to_csv(output_file, index=False)
    print(f"\nResults saved to: {output_file}")
    
    # Display the full results dataframe
    print("\nFull Results DataFrame:")
    display(psi_results_df)
else:
    print("No PSI results generated. Please check your data structure.")

Starting PSI Analysis with Quantile-Based Binning...
Initial data overview:
Total records: 542279
Columns: ['customerId', 'digitalLoanAccountId', 'modelDisplayName', 'modelVersionId', 'prediction_score', 'appln_submit_datetime', 'disbursementDateTime']
Models: ['Beta-Cash-Stack-Model' 'Beta-Cash-AppScore-Model' 'Beta-Cash-Demo-Model'
 'Alpha-Cash-CIC-Model' 'Alpha-Cash-Stack-Model'
 'Alpha  - IncomeEstimationModel' 'Alpha - CIC-SIL-Model'
 'Alpha - StackingModel' 'Beta - AppsScoreModel' 'Beta - DemoScoreModel'
 'Beta - IncomeEstimationModel' 'Beta - StackScoreModel']

Prediction column info:
Data type: float64
Non-null values: 542279
Null values: 0
Prediction sample values: [0.1330928089191726, 0.3564340397928232, 0.4613876570074397, 0.13208198157007744, 0.3543057352141376]
Prediction range: 0.0067146935 - 158750.78884243715
Prediction mean: 3844.4776977540705
Prediction median: 0.18583161176314333

Converting datetime...
Removed 1606 records with invalid dates
Processing predictions..

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_33404\4046866132.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baseline_df['prediction_bin'] = pd.cut(
C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_33404\4046866132.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baseline_df['prediction_bin'] = pd.cut(
C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_33404\4046866132.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co


Month 2025-09:
  PSI = 0.000000
  Records = 803
  Prediction stats: min=0.11, max=0.98, mean=0.62

Month 2025-10:
  PSI = 0.014096
  Records = 1279
  Prediction stats: min=0.05, max=0.99, mean=0.62

Processing model: Beta-Cash-Demo-Model
Months available: ['2025-09', '2025-10']
Records per month:
  2025-09: 802 records
  2025-10: 1279 records

Baseline month: 2025-09, Records: 802
Baseline prediction stats:
  Min: 0.248151
  Max: 0.713393
  Mean: 0.557510
  Median: 0.572025

Quantile bin edges: [0.24805123 0.44883072 0.48210136 0.51352534 0.5453822  0.57202458
 0.58956251 0.60412577 0.63118598 0.66526366 0.71349342]
Baseline distribution across bins:
  Bin 0: 0.1010 (81 records)
  Bin 1: 0.1010 (81 records)
  Bin 2: 0.0985 (79 records)
  Bin 3: 0.0998 (80 records)
  Bin 4: 0.0998 (80 records)
  Bin 5: 0.0998 (80 records)
  Bin 6: 0.0998 (80 records)
  Bin 7: 0.0998 (80 records)
  Bin 8: 0.1010 (81 records)
  Bin 9: 0.0998 (80 records)

Month 2025-09:
  PSI = 0.000000
  Records = 802
 

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_33404\4046866132.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baseline_df['prediction_bin'] = pd.cut(
C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_33404\4046866132.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baseline_df['prediction_bin'] = pd.cut(
C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_33404\4046866132.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co


Month 2025-09:
  PSI = 0.553104
  Records = 21413
  Prediction stats: min=0.01, max=0.39, mean=0.07

Month 2025-10:
  PSI = 0.615713
  Records = 5675
  Prediction stats: min=0.01, max=0.31, mean=0.07

Processing model: Beta - IncomeEstimationModel
Months available: ['2025-03', '2025-04', '2025-05', '2025-06', '2025-07', '2025-08', '2025-09', '2025-10']
Records per month:
  2025-03: 414 records
  2025-04: 1039 records
  2025-05: 1561 records
  2025-06: 16368 records
  2025-07: 22790 records
  2025-08: 23270 records
  2025-09: 21413 records
  2025-10: 5673 records

Baseline month: 2025-03, Records: 414
Baseline prediction stats:
  Min: 7724.886081
  Max: 60822.050743
  Mean: 15926.646706
  Median: 9183.843452

Quantile bin edges: [  6070.44295969   8539.50316933   8856.7180479    8957.17789463
   9067.55365474   9183.84345241   9357.09627399  25685.68611341
  27627.96431343  29863.25393966 158750.78894244]
Baseline distribution across bins:
  Bin 0: 0.1014 (42 records)
  Bin 1: 0.0990 (

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_33404\4046866132.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baseline_df['prediction_bin'] = pd.cut(


,modelDisplayName,month,baseline_month,PSI,account_count,sample_size,prediction_min,prediction_max,prediction_mean,num_bins,psi_category
0,Beta-Cash-Stack-Model,2025-09,2025-09,0.000000,802,803,0.114708,0.982625,0.616184,10,Green
1,Beta-Cash-Stack-Model,2025-10,2025-09,0.014096,1279,1279,0.045575,0.985519,0.618677,10,Green
2,Beta-Cash-Demo-Model,2025-09,2025-09,0.000000,802,802,0.248151,0.713393,0.557510,10,Green
3,Beta-Cash-Demo-Model,2025-10,2025-09,0.038174,1279,1279,0.271093,0.713393,0.551189,10,Green
4,Beta-Cash-AppScore-Model,2025-09,2025-09,0.000000,521,522,0.304277,0.781791,0.542615,10,Green
5,Beta-Cash-AppScore-Model,2025-10,2025-09,0.021127,861,861,0.258662,0.811925,0.544188,10,Green
6,Alpha-Cash-CIC-Model,2025-09,2025-09,0.000000,134,134,0.285958,0.758973,0.512970,10,Green
7,Alpha-Cash-CIC-Model,2025-10,2025-09,0.146695,202,202,0.289370,0.765633,0.503485,10,Amber
8,Alpha-Cash-Stack-Model,2025-09,2025-09,0.000000,134,134,0.181000,0.814357,0.470372,10,Green
9,Alpha-Cash-Stack-Model,2025-10,2025-09,0.054775,202,202,0.055939,0.815492,0.452540,10,Green


In [31]:
psi_results_df.sort_values(['modelDisplayName', 'month']).to_csv(r"D:\OneDrive - Tonik Financial Pte Ltd\MyStuff\Data Engineering\Model_Monitoring\New_Model_Monitoring\Data\PSI_values.csv", index = False)

In [32]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.temp_PSI_new_monitoring_dc"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results_df, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(
C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:622: UserWarning: Pyarrow could not determine the type of columns: psi_category.
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=cc573826-287f-43b1-950e-d4abeaae7157>